In [2]:
# IMDB exploration
import pandas as pd

# import name.basics.tsv
name_basics = pd.read_csv('name.basics.tsv', sep='\t')
name_basics.head()
# re-index on nconst
name_basics.set_index('nconst', inplace=True)

# import title.basics.tsv
title_basics = pd.read_csv('title.basics.tsv', sep='\t')
title_basics.head()
# re-index on tconst
title_basics.set_index('tconst', inplace=True)

# import title.principals.tsv
title_principals = pd.read_csv('title.principals.tsv', sep='\t')
title_principals.head()
# re-index on tconst
title_principals.set_index('tconst', inplace=True)

# import title.ratings.tsv
title_ratings = pd.read_csv('title.ratings.tsv', sep='\t')
title_ratings.head()
# re-index on tconst
title_ratings.set_index('tconst', inplace=True)

# import title.crew.tsv
title_crew = pd.read_csv('title.crew.tsv', sep='\t')
title_crew.head()
# re-index on tconst
title_crew.set_index('tconst', inplace=True)

# import title.akas.tsv
title_akas = pd.read_csv('title.akas.tsv', sep='\t')
title_akas.head()
# re-index on titleId
title_akas.set_index('titleId', inplace=True)

# import title.episode.tsv
title_episode = pd.read_csv('title.episode.tsv', sep='\t')
title_episode.head()
# re-index on tconst
title_episode.set_index('tconst', inplace=True)

C:\Users\Dave\AppData\Local\Temp\ipykernel_24356\326601405.py:11: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  title_basics = pd.read_csv('title.basics.tsv', sep='\t')


In [3]:
# ok now add all of theses dataframes to SQL database
import sqlite3
conn = sqlite3.connect('imdb.db')
name_basics.to_sql('name_basics', conn)
title_basics.to_sql('title_basics', conn)
title_principals.to_sql('title_principals', conn)
title_ratings.to_sql('title_ratings', conn)
title_crew.to_sql('title_crew', conn)
title_akas.to_sql('title_akas', conn)
title_episode.to_sql('title_episode', conn)

# now close the connection
conn.close()


